# core

> Fill in a module description here

In [1]:
#| default_exp core

In [1]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
import toml
import os
from pathlib import Path
from dotenv import load_dotenv
import re

load_dotenv()
def set_default_dir():
    print('Setting reproduce.work config dir to ./reproduce')
    return Path("./reproduce")

reproduce_dir = os.getenv("REPROWORKDIR", set_default_dir())
dev_image_tag = os.getenv("REPRODEVIMAGE")

def read_base_config():
    with open(Path(reproduce_dir, 'config.toml'), 'r') as f:
        base_config = toml.load(f)
    return base_config


def generate_config():
    # Authors section
    author1_email = input("Enter author's email (required): ")
    author1_name = input("Enter author's name: ")
    author1_affiliation = input("Enter author's affiliation: ")

    repro_version = input("Enter reproduce_work API version (Default: 'reproduce.work/v1/default'): ") or "reproduce.work/v1/default"
    
    
    if repro_version in ["reproduce.work/v1/default"]:
        document_dir = reproduce_dir
        input_file = f"{document_dir}/main.md"
        dynamic_file = f"{document_dir}/data.toml"
        latex_template = f"{document_dir}/latex/template.tex"
        output_file =f"{document_dir}/latex/compiled.tex"
        watch_files = [input_file, dynamic_file, latex_template]
        
    else:
        document_dir = input(f"Enter document directory (Default: {reproduce_dir}): ") or reproduce_dir
        input_file = input(f"Enter input file path (Default: {document_dir}/main.md): ") or f"{document_dir}/main.md"
        dynamic_file = input(f"Enter dynamic file path (Default: {document_dir}/data.toml): ") or f"{document_dir}/data.toml"
        latex_template = input(f"Enter latex template path (Default: {document_dir}/latex/template.tex): ") or f"{document_dir}/latex/template.tex"
        output_file = input(f"Enter output file path (Default: {document_dir}/latex/compiled.tex): ") or f"{document_dir}/latex/compiled.tex"
        watch_files_input = input(f"Enter files to watch (comma-separated, Default: {document_dir}/main.md,{document_dir}/data.toml,{document_dir}/latex/template.tex): ")
        watch_files = watch_files_input.split(",") if watch_files_input else [f"{document_dir}/main.md", f"{document_dir}/data.toml", f"{document_dir}/latex/template.tex"]
    
    # Repro stage init section
    dev_image_tag = input("Enter dev image tag (required; the docker image of your local development workflow): ")
    
    # Check for critical fields
    if not dev_image_tag:
        print("Error: Missing critical field 'dev_image_tag'.")
        return
    
    #reproduce_dir_default = f"{reproduce_dir}"  # this can be changed if needed
    #reproduce_dir = input(f"Enter reproduce directory (Default: {reproduce_dir_default}): ") or reproduce_dir_default
    
    repro_init_script = f'''docker build -t {dev_image_tag} .
docker build -t tinytex {reproduce_dir}/Dockerfile.tinytex
docker build -t watcher {reproduce_dir}/Dockerfile.watch'''
    
    newline = "\n"
    with open(Path(reproduce_dir, 'config.toml'), 'w') as f:
        f.write(f'''
[authors]
author1.email = "{author1_email}"{newline + 'author1.name = "' + author1_name + '"' if author1_name else ""}{newline + 'author1.affiliation = "' + author1_affiliation + '"' if author1_affiliation else ""}

# reproduce.work configuration
[repro]
version = "{repro_version}"
stages = ["init", "develop", "build"]

[repro.files]
input = "{input_file}"
dynamic = "{dynamic_file}"
latex_template = "{latex_template}"
output = "{output_file}"
watch = {watch_files}

[repro.stage.init]
script = """
{repro_init_script}
"""

[repro.stage.develop]
script = """
docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag}
\INSERT{{watch_cmd_here}}
"""

[repro.stage.build]
script = """
docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of INSERTvar in input file
docker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex
docker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/{document_dir}/latex && xelatex compiled.tex" # this compiles the latex
"""
''')


    # By default, TOML files cannot reference variables defined earlier in the file
    # This is a hack to get around that which allows me to read in the TOML data,
    # and then backfill the watch command with the files listed in the TOML file itself.
    if '\INSERT{watch_cmd_here}' in open(Path(reproduce_dir, 'config.toml'), 'r').read():
        with open(Path(reproduce_dir, 'config.toml'), 'r') as f:
            base_config_str = f.read()
        replaced_config = base_config_str.replace(
            '\INSERT{watch_cmd_here}', 
            f"""docker run watcher "{','.join(watch_files)}" "echo \"File has changed!\" && build_cmd" """
        )
        with open(Path(reproduce_dir, 'config.toml'), 'w') as f:
            f.write(replaced_config)
    else:
        print('did not replace watch command')


def update_watched_files(add=[], remove=[]):
    base_config = read_base_config()
    existing_files = base_config['repro']['files']['watch']
    new_files = existing_files + [a for a in add if a not in existing_files]
    new_files = [f for f in new_files if f not in remove]
    base_config['repro']['files']['watch'] = new_files

    current_develop_script = base_config['repro']['stage']['develop']['script']
    current_develop_script
    # regex to replace content in string matching 'watcher \"{to_replace}\"'
    # with 'watcher \"{new_files}\"'
    # and replace 'build_cmd' with 'python reproduce_work.build()'
    import re
    new_develop_script = re.sub(
        r'watcher \"(.*?)\"', 
        f'watcher \"{",".join(new_files)}\"', 
        current_develop_script
    )
    base_config['repro']['stage']['develop']['script'] = new_develop_script

    with open(Path(reproduce_dir, 'config.toml'), 'w') as f:
        toml.dump(base_config, f)
        
    print(f"Updated watched files to {new_files}")
    return new_files


def validate_base_config(base_config):
    required_keys = ['authors', 'repro']
    for key in required_keys:
        if key not in base_config:
            print(toml.dumps(base_config))
            print(f"Error: Missing required field '{key}' in config.toml")
            return False
        if key=='repro':
            if 'stages' not in base_config['repro']:
                print(f"Error: Missing required field 'repro.stages' in reproduce.work configuration at {reproduce_dir}/config.toml")
                return False
            for stage in base_config['repro']['stages']:
                if (f'repro.stage.{stage}' not in base_config) and (stage not in base_config['repro']['stage']):
                    print(toml.dumps(base_config))
                    print(f"Error: Missing required field repro.stage.{stage} in reproduce.work configuration at {reproduce_dir}/config.toml")
                    return False
    return True

def requires_config(func):
    def wrapper(*args, **kwargs):
        config = read_base_config()
        if not validate_base_config(config):
            raise Exception("Your reproduce.work configuration is not valid.")
        return func(*args, **kwargs)
    return wrapper

# Write some basic tests
def test_validate_base_config():
    # Test a valid base_config
    base_config = {
        'authors': {
            'author1.name': 'Alex P. Miller',
            'author1.affiliation': 'USC Marshall School of Business',
            'author1.email': 'alex.miller@marshall.usc.edu'
        },
        'repro': {
            'version': 'reproduce.work/v1/default',
            'stages': ['init', 'develop', 'build']
        },
        'repro.files': {
            'input': 'document/main.md',
            'dynamic': 'document/data.toml',
            'latex_template': 'document/latex/template.tex',
            'output': 'document/latex/compiled.tex',
            'watch': ['document/main.md', 'document/data.toml', 'document/latex/template.tex']
        },
        'repro.stage.init': {
            'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
        },
        'repro.stage.develop': {
            'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
        },
        'repro.stage.build': {
            'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
        }
    }
    assert validate_base_config(base_config) == True
    
    # Test an invalid base_config
    base_config = {
        'authors': {
            'author1.name': 'Alex P. Miller',
            'author1.affiliation': 'USC Marshall School of Business',
            'author1.email': 'alex.miller@marshall.usc.edu'
        },
        'repro': {
            'version': 'reproduce.work/v1/default',
            'stages': ['init', 'develop', 'build']
        },
        'repro.files': {
            'input': 'document/main.md',
            'dynamic': 'document/data.toml',
            'latex_template': 'document/latex/template.tex',
            'output': 'document/latex/compiled.tex'
        },
        'repro.stage.init': {
            'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
        },
        'repro.stage.develop': {
            'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
        },
        'repro.stage.build': {
            'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
        }
    }
    assert validate_base_config(base_config) == False
    
    print("All tests passed!")



Setting reproduce.work config dir to ./reproduce


In [78]:
#import os
#os.listdir(os.getcwd())

In [79]:
base_config = {
        'authors': {
            'author1.name': 'Alex P. Miller',
            'author1.affiliation': 'USC Marshall School of Business',
            'author1.email': 'alex.miller@marshall.usc.edu'
        },
        'repro': {
            'version': 'reproduce.work/v1/default',
            'stages': ['init', 'develop', 'build']
        },
        'repro.files': {
            'input': 'document/main.md',
            'dynamic': 'document/data.toml',
            'latex_template': 'document/latex/template.tex',
            'output': 'document/latex/compiled.tex'
        },
        'repro.stage.init': {
            'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
        },
        'repro.stage.develop': {
            'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
        },
        'repro.stage.build': {
            'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
        }
    }
validate_base_config(base_config)

True

In [80]:
#| hide
import nbdev; nbdev.nbdev_export()